<a href="https://colab.research.google.com/github/Minseo-202103621/machinelearning_deeplearning/blob/main/CH)07_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 심층 신경망
## 인공 신경망에 층을 여러 개 추가하여 패션 MNIST 데이터 셋을 분류하면서 케라스로 심층 신경망을 만드는 방법을 자세히 배운다.

1절에서 만들었던 인공 신경망의 성능을 더 높여 보자

---


# 2개의 층

* 다시 케라스 API를 사용해서 패션 MNIST 데이터 셋을 불러오겠다.

In [1]:
from tensorflow import keras
(train_input,train_target),(test_input,test_target) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


* 그 다음 이미지의 픽셀값을 0 ~ 255 범위에세 0~1 사이로 변환하고, 28 X 28 크기의 2차원 배열을 784 크기의 1차원 배열로 펼친다.
* 그 다음 사이킷런의 train_test_split() 함수로 훈련 세트와 검증 세트로 나눈다.

In [2]:
from sklearn.model_selection import train_test_split
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1,28*28)
train_scaled,val_scaled, train_target, test_target = train_test_split(train_scaled,train_target,test_size = 0.2, random_state=42)


이제 인공 신경망 모델에 층을 2개 추가해 보겠다. 여기서 만들 모델의 대략적인 구조는 다음 그림과 같다.






1절에서 만든 신경망 모델과 다른 점은 입력층과 출력층 사이에 밀집층이 추가된 것이다. 이렇게 입력층과 출력층 사이에 있는 모든 층을 ***은닉층***이라고 부른다.

은닉층에는 주황색 원으로 활성화 함수가 표시되어 있다. 활성화 함수는 신경망 층의 선형 방정식의 계산 값에 적용하는 함수이다. 이전 절에서 출력층에 적용했던 소프트맥스 함수도 활성화 함수이다. 

출력층에 적용하는 활성화 함수는 종류가 제한되어 있다. 이진 분류는 시그모이드 다중 분류는 소프트맥스 함수를 사용한다.

이에 비해 은닉층의 활성화 함수는 비교적 자유롭다. 대표적으로 시그모이드 함수와 렐루 함수 등을 사용한다.

** 회귀를 위한 신경망의 출력층에서는 활성화 함수를 적용할 필요가 없다. 분류 문제는 클래스에 대한 확률을 출력하기 위해 활성화 함수를 사용한다. 하지만 회귀의 출력은 임의의 어떤 숫자이므로 활성화 함수를 적용할 필요가 없다. 즉 출력층의 선형 방정식의 계산을 그대로 출력한다. 이렇게 하려면 DENSE 층의 activation 매개변수에 아무런 값을 지정하지 않는다.**






그런데 은닉층에 왜 활성화 함수를 적용할까?? 

a X 4 + 2 = b        b X 3 - 5 = c

=> a X 12 + 1 = c

이 2개의 선형 방정식이 합쳐지면 b는 결국 아무런 일도 하지 않고 사라지게 되는걸 볼 수 있다.

신경망도 마찬가지이다. 은닉층에서 선형적인 산술 계산만 수행한다면 수행 역할이 없는 셈이다. 선형 계산을 적당하게 비선형적으로 비틀어 주어야 한다. 그래야 다음 층의 계산과 단순히 합쳐지지 않고 나름의 역할을 할 수 있다.

마치  a X 4 + 2 = b     log(b) = k   =>  k X 3 - 5 = c 

많이 사용하는 활성화 함수 중 하나는 4장에서 배웠던 시그모이드 함수이다. 

이함수는 뉴런의 출력 z값을 0과 1사이로 압축한다.

* 그럼 시그모이드 활성화 함수를 사용한 은닉층과 소프트맥스 함수를 사용한 출력층을 케라스의 Dense 클래스로 만들어 보겠다. 
* 케라스에서 신경망의 첫 번째 층은 input_shape 매개변수로 입력의 크기를 꼭 지정해 주어야 한다.





In [3]:
dense1 = keras.layers.Dense(100,activation='sigmoid',input_shape=(784,))
dense2 = keras.layers.Dense(10,activation = 'softmax')


dense1이 은닉층이고 100개의 뉴런을 가진 밀집층이다. 활성화 함수를 sigmoid로 지정했고 input_shape 매개변수에서 입력의 크기를 (784,)로 지정했다. 은닉층이 뉴런 개수를 정하는 데는 특별한 기준이 없다. 몇 개의 뉴런을 두어야 할지 판단하기 위해서는 상당한 경험이 필요하다.

여기서 한 가지 제약 사항이 있다면 적어도 출력층의 뉴런보다는 많게 만들어야 한다.

클래스 10개에 대한 확률을 예측해야 하는데 이전 은닉층의 뉴런이 10개보다 적다면 부족한 정보가 전달될 것이다. 

그 다음 dense2는 출력층이다. 10개의 클래스를 분류하므로 10개의 뉴런을 두었고 활성화 함수는 소프트맥스 함수로 지정했다.

---

# 심층 신경망 만들기

* 이제 앞에서 만든 dense1과 dense2 객체를 Sequential 클래스에 추가하여 심층 신경망(deep neural network, DNN)을 만들어 보겠다.

In [4]:
model = keras.Sequential([dense1,dense2])

Sequential 클래스의 객체를 만들 때 여러 개의 층을 추가하려면 이와 같이 dense1과 dense2를 리스트로 만들어 전달한다.

여기서 주의할 것은 출력층을 가장 마지막에 두어야 한다는 것이다.

이 리스트는 가장 처음 등장하는 은닉층에서 마지막 출력층의 순서로 나열해야 한다.

(그림)

인공 신경망의 강력한 성능은 바로 이렇게 층을 추가하여 입력 데이터에 대해 연속적인 학습을 진행하는 능력에서 나온다. 

앞에서 공부한 선형 회귀, 로지스틱 회귀, 결정 트리 등 다른 머신러닝 알고리즘과 대조된다.

케라스는 모델의 summary() 메서드를 호출하면 층에 대한 유용한 정보를 얻을 수 있다.


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


층마다 층 이름, 클래스, 출력 크기, 모델 파라미터 개수가 출력된다. 층을 만들 때 name 매개변수로 이름을 지정할 수 있다. 층 이름을 지정하지 않으면 케라스가 자동으로 dense라고 이름을 붙인다.

출력 크기를 보면 (None,100)이다. 첫 번째 차원은 샘플의 개수를 나타낸다. 샘플 개수가 아직 정의되어 있지 않기 때문이다. 왜 그러느냐...? 케라스 모델의 fit() 메서드에 훈련 데이터를 주입하면 이 데이터를 한 번에 모두 사용하지 않고 잘게 나누어 여러 번에 걸쳐 경사 하강법 단계를 수행한다. 미니배치 경사 하강법을 사용하는 거다. 

케라스의 기본 미니배치 크기는 32개이다. 이 값은 fit() 메서드에서 batch_size 매개변수로 바꿀 수 있다. 따라서 샘플 개수를 고정하지 않고 어떤 배치 크기에도 유연하게 대응할 수 있도록 None으로 설정한다.

은닉층의 뉴런 개수를 100개로 두었으니 100개의 출력이 나온다. 즉 샘플마다 784개의 픽셀값이 은닉층을 통과하면서 100개의 특성으로 압축되었다.

마지막으로 모델 파라미터 개수가 출력된다. 이 층은 Dense 층이므로 입력 픽셀 784개와 100개의 모든 조합에 대한 가중치가 있다. 그리고 뉴런마다 1개의 절편이 있다.  784 X 100 + 100 = 78,500개이다. 

두 번째 층은 100 X 10 + 10 = 1010개이다.

---
 
# 층을 추가하는 다른 방법

모델을 훈련하기 전에 Sequential 클래스에 층을 추가하는 다른 방법을 알아보겠다. 

앞에서는 dense를 만들어 시퀀스 클래스에 전달했지만 이 두 객체를 따로 저장하여 쓸 일이 없기 때문에 다음처럼 Sequential 클래스의 생성자 안에서 바로 Dense 클래스의 객체를 만드는 경우가 많다.

In [6]:
model = keras.Sequential([keras.layers.Dense(100, activation='sigmoid',input_shape=(784,),name='hidden'),
keras.layers.Dense(10,activation='softmax',name='output')
],name = '패션 MNIST 모델')

모델의 이름과 달리 층의 이름은 반드시 영문이어야 한다.

In [7]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


이 방법이 편리하지만 아주 많은 층을 추가하려면 Sequential 클래스 생성자가 매우 길어진다. 또 조건에 따라 층을 추가할 수도 없다. 

Sequential클래스에서 층을 추가할 때 가장 널리 사용하는 방법은 모델의 add() 메서드이다.

* 이 방법은 다음처럼 Sequential 클래스의 객체를 만들고 이 객체의 add() 메서드를 호출하여 층을 추가한다.


In [8]:
model = keras.Sequential()
model.add(keras.layers.Dense(100,activation='sigmoid',input_shape=(784,)))

model.add(keras.layers.Dense(10,activation='softmax'))

여기에서도 Dense 클래스의 객체를 따로 변수에 담지 않고 바로 add() 메서드로 전달한다. 이 방법은 한눈에 추가되는 층을 볼 수 있고 프로그램 실행 시 동적으로 층을 선택하여 추가할 수 있다.


In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


이제 모델을 훈련해 보겠다.
* compile() 메서드 설정은 이전과 동일하고 여기서도 5번의 에포크 동안 훈련해보겠다.



In [10]:
model.compile(loss='sparse_categorical_crossentropy',metrics = 'accuracy')
model.fit(train_scaled,train_target,epochs=5)

Epoch 1/5
1500/1500 [==============================] - 9s 5ms/step - loss: 0.5689 - accuracy: 0.8050
Epoch 2/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4101 - accuracy: 0.8534
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3761 - accuracy: 0.8641
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3549 - accuracy: 0.8714
Epoch 5/5
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3363 - accuracy: 0.8794


훈련 세트에 대한 성능을 보면 추가된 층이 성능을 향상시켰다는 것을 잘 알 수 있다.

다음에는 이미지 분류에서 높은 성능을 내는 활성화 함수에 대해 알아보겠다.


---


# 렐루 함수

초창기 인공 신경망 은닉층에 많이 사용된 활성화 함수는 시그모이드 함수이다. 하지만 시그모이드 그래프를 보면 오르쪽과 왼쪽 끝으로 갈수록 그래프가 누워있기 때문에 올바른 출력을 만드는데 신속하게 대응하지 못한다. 

특히 층이 많은 심층 신경망일수록 그 효과가 누적되어 학습은 더 어렵게 만든다. 

이를 개선하기 위해 다른 종류의 활성화 함수가 제안됐다. 바로 ***렐루ReLU***함수이다. 

렐루 함수는 입력이 양수일 경우 마치 활성화 함숙 없는 것처럼 그냥 입력을 통과시키고 음수일 경우에는 0으로 만든다.

렐루 함수는 max(0,z)와 같이 쓸 수 있다. 이 함수는 z가 0보다 작으면 0을 아니라면 z를 출력하는 함수를 나타낸다.  렐루 함수는 특히 이미지 처리에서 좋은 성능을 낸다고 알려져 있다.

이 렐루 함수를 적용하기 전에 케라스에서 제공하는 편리한 층 하나를 더 살펴보겠다.

패션 MNIST 데이터는 28 X 28 크기여서 인공 신경망에 주입하기 전에 reshape() 메서드를 사용해 1차원으로 펼친다. 직접 이렇게 펼쳐도 되지만 케라스에서는 이를 위한 Flatten 층을 제공한다.

사실 Flatten 클래스는 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할만 한다. 입력에 곱해지는 가중치나 절편이 없다. 따라서 인공 신경망의 성능을 위해 기여하는 바는 없다. 

In [11]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100,activation = 'relu'))
model.add(keras.layers.Dense(10,activation='softmax'))


In [12]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


첫 번째 등장하는 Flatten 클래스에 포함된 모델 파라미터는 0개이다. 케라스의 Flatten 층을 신경망 모델에 추가하면 입력값의 차원을 짐작할 수 있는 것이 또 하나의 장점이다. 앞의 출력에서 784개의 입력이 첫 번째 은닉층에 전달되는 것을 알 수 있다.

* 이제 훈련 데이터를 다시 준비해서 모델을 훈련해 보겠다.

In [13]:
(train_input,train_target),(test_input,test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled,val_scaled,train_target,val_target = train_test_split(train_scaled,train_target,test_size=0.2,random_state=42)

* 모델을 컴파일하고 훈련하는 것은 다음 코드처럼 이전과 동일하다.

In [14]:
model.compile(loss = 'sparse_categorical_crossentropy',metrics='accuracy')
model.fit(train_scaled,train_target,epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5341 - accuracy: 0.8117
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3952 - accuracy: 0.8569
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3552 - accuracy: 0.8714
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3322 - accuracy: 0.8806
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3160 - accuracy: 0.8866


시그모이드 함수를 사용했을 때와 비교하면 성능이 조금 향샹되었다. 크진 않지만 렐루 함수의 효과를 봤다. 

* 그럼 검증 세트에서의 성능도 확인해 보겠다.



In [15]:
model.evaluate(val_scaled,val_target)


375/375 [==============================] - 1s 2ms/step - loss: 0.3469 - accuracy: 0.8786


[0.34693610668182373, 0.8785833120346069]

1절의 은닉층을 추가하지 않은 경우보다 몇 퍼센트 성능이 향샹됐다. 지금까지는 모델을 5번의 에포크 동안 훈련했다. 이보다 더 훈련하지 않을 이유는 없다.

그전에 인공 신경망의 하이퍼파라미터에 대해 잠시 알아보고 이번 절을 마무리하겠다.

---

# 옵티마이저

하이퍼파라미터는 모델이 학습하지 않아 사람이 지정해 주어야 하는 파라미터이다. 신경망에는 특히 하이퍼파라미터가 많다. 

이번 절에서 배운 은닉층. 여러 개의 은닉층을 추가할 수도 있고 하나만 추가할 수도 있다. 추가할 은닉층의 개수는 모델이 학습하는 것이 아니라 우리가 지정해 주어야 할 하이퍼파라미터이다. 은닉층의 뉴런 개수도 활성화 함수도 층의 종류도 다 하이퍼파라마티이다.(밀집층말고 다른 종류의 층을 선택할 수도 있다.)

케라스는 기본적으로 미니배치 경사 하강법을 사용하며 미니배치 개수는 32개이다. fit()메서드의 batch_size 매개변수에서 이를 조정할 수 있으며 역시 하이퍼파라미터이다. 또한 fit()메서드의 epochs 매개변수도 하이퍼파라미터이다. 반복 횟수에 따라 다른 모델이 만들어진다.

마지막으로 compile() 메서드에서는 케라스의 기본 경사 하강법 알고리즘인 RMSprop을 사용했다. 케라스는 다양한 종류의 경사 하강법 알고리즘을 제공한다.
이들을 ***옵티마이저***라고 부른다. 다른 옵티마이저도 테스트 해보겠다. 또한 RMSprop의 학습률 또한 조정할 하이퍼파라미터 중 하나이다.

이처럼 정말 많기 때문에 처음부터 모델을 구성하고 각종 하이퍼파라미터의 최적값을 찾는 것은 어려운 작업이다. 여기서는 여러 가지 옵티마이저를 테스트해 보겠다. 

가장 기본적인 옵티마이저는 확률적 경사 하강법인 SGD이다. 이름이 SGD이지만 1개의 샘플을 뽑아서 훈련하지 않고 앞서 언급한 것처럼 기본적으로 미니배치를 사용한다.

* SGD 옵티마이저를 사용하려면 compile() 메서드의 optimizer 매개변수를 'sgd'로 지정한다.

In [16]:
model.compile(optimizer='sgd',loss = 'sparse_categorical_crossentropy',metrics = 'accuracy')

이 옵티마이저는 tensorflow.keras.optimizers 패키지 아래 SGD 클래스로 구현되어 있다.

'sgd' 문자열은 이 클래스의 기본 설정 매개변수로 생성한 객체와 동일하다. 즉 다음 코드는 위의 코드와 정확히 동일한다.



In [17]:
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd,loss = 'sparse_categorical_crossentropy',metrics = 'accuracy')

sgd ?? 'sgd' ??

원래 sgd = keras.optimizers.SGD()처럼 SGD 클래스 객체를 만들어 사용해야 하는데 번거로움을 피하고자 'sgd'라 지정하면 자동으로 SGD 클래스 객체를 만들어 준다.

만약 SGD 클래스의 학습률 기본값이 0.01일 때 이를 바꾸고 싶다면 다음과 같이 원하는 학습률을 learning_rate 매개변수에 지정하여 사용한다.



In [18]:
sgd = keras.optimizers.SGD(learning_rate=0.1)


SGD외에도 다양한 옵티마이저들이 있다. 

기본 경사 하강법 옵티마이저는 모두 SGD 클래스에서 제공한다. SGD 클래스의 momentum 매개변수의 기본값은 0이다. 이를 0보다 큰 값으로 지정하면 마치 이전의 그레이디언트를 가속도처럼 사용하는 **모멘텀 최적화**를 사용한다. 보통 momentum 매개변수는 0.9 이상을 지정한다.

다음처럼 SGD 클래스의 nesterov 매개변수를 False에서 True로 바꾸면 **네스테로프 모멘텀 최적화**(또는 네스테로프 가속 경사)를 사용한다.



In [19]:
sgd = keras.optimizers.SGD(momentum=0.9,nesterov = True)


네스테로프 모멘텀은 모멘텀 최적화를 2번 반복하여 구현한다. 대부분의 경우 네스테로프 모멘텀 최적화가 기본 확률적 경사 하강법보다 더 나은 성능을 제공한다.

모델이 최적점에 가까이 갈수록 학습률을 낮출 수 있다. 이렇게 하면 안정적으로 최적점에 수렴할 가능성이 높다. 이런 학습률을 **적응적 학습률**이라고 한다.
이런 방식들은 학습률 매개변수를 튜닝하는 수고를 덜 수 있는 것이 장점이다.

적응적 학습률을 사용하는 대표적인 옵티마이저는 Adagrad와 RMSprop이다. 각각 compile() 메서드의 optimizer 매개변수에 'adagrad'와 'rmsprop'으로 지정할 수 있다. optimizer 매개변수의 기본값이 바로 'rmsprop'이다. 이 두 옵티마이저의 매개변수를 바꾸고 싶다면 SGD와 같이 Adagrad와 RMSprop 클래스 객체를 만들어 사용하면 된다.

In [20]:
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer = adagrad,loss = 'sparse_categorical_crossentropy')



* RMSprop도 마찬가지이다.

In [21]:
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer = rmsprop,loss='sparse_categorical_crossentropy',metrics='accuracy')


모멘텀 최적화와 RMSprop의 장점을 접목한것이 Adam이다. Adam은 RMSprop과 함께 맨처음 시도해 볼 수 있는 좋은 알고리즘이다. Adam 클래스도 keras.optimizers 패키지 아래에 있다. 

적응적 학습률을 사용하는 이 3개의 클래스는 learning_rate 매개변수의 기본값으로 모두 0.001사용한다.

* 여기서는 Adam 클래스의 매개변수 기본값을 사용해 패션 MNIST 모델을 훈련해 보겠다.

In [22]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dense(10,activation = 'softmax'))


* compile() 메서드의 optimizer를 'adam'으로 설정하고 5번의 에포크 동안 훈련한다.

In [23]:
model.compile(optimizer='adam',loss = 'sparse_categorical_crossentropy',metrics='accuracy')
model.fit(train_scaled,train_target,epochs=5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5383 - accuracy: 0.8113
Epoch 2/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3994 - accuracy: 0.8585
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3586 - accuracy: 0.8702
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3298 - accuracy: 0.8794
Epoch 5/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3090 - accuracy: 0.8866


In [24]:
model.evaluate(val_scaled,val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3538 - accuracy: 0.8708


[0.3537600040435791, 0.8708333373069763]